In [1]:
cd RTM/src/lib

/home/kaixin1/Project/RTM/src/lib


In [36]:
from models.losses import FocalLoss, L1Loss, BinRotLoss
from models.decode import ddd_decode
from models.utils import _sigmoid
from utils.debugger import Debugger
from utils.post_process import ddd_post_process
from utils.oracle_utils import gen_oracle_map
from models.losses import FocalLoss, RegL1Loss, RegLoss, RegWeightedL1Loss

In [3]:
cd ../

/home/kaixin1/Project/RTM/src


In [4]:
from models.utils import _transpose_and_gather_feat

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [6]:
import torch
torch.__version__

'0.4.1.post2'

In [7]:
import os
import time
import math
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchsummary import summary

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

In [9]:
args = ['ddd', '--exp_id', '3dop', '--dataset', 'kitti',\
'--kitti_split', '3dop' ,'--batch_size' ,'16' ,'--master_batch', '7' ,'--num_epochs','70',\
'--arch','resFP_18','--lr_step', '45,60' ,'--gpus', '0,1']

In [10]:
opt = opts().parse(args)

Fix size testing.
training chunk_sizes: [7, 9]
The output will be saved to  /home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop


In [11]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
# Dataset多继承了数据库类(KITTI)和任务类(ddd)；
Dataset = get_dataset(opt.dataset, opt.task)
# 在参数里添加与用到的数据集的相关属性
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)

heads {'hm': 3, 'vertex_hm': 9, 'vertex_coordinate': 18, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2, 'vertex_reg_offset': 2}


In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

In [13]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(\
  model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth
=> init deconv weights from normal distribution


In [14]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)

In [15]:
trainer.loss

DddLoss(
  (crit): FocalLoss()
  (crit_reg): L1Loss()
  (crit_rot): BinRotLoss()
)

In [16]:
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [17]:
val_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'val'), 
      batch_size=1, 
      shuffle=False,
      num_workers=1,
      pin_memory=True
  )

==> initializing kitti 3dop, val data.
loading annotations into memory...
Done (t=0.83s)
creating index...
index created!
Loaded val 3769 samples


In [18]:
self = Dataset(opt, 'val')

==> initializing kitti 3dop, val data.
loading annotations into memory...
Done (t=0.88s)
creating index...
index created!
Loaded val 3769 samples


In [19]:
import torch.utils.data as data
import pycocotools.coco as coco
import numpy as np
import torch
import json
import cv2
import os
import math
from utils.image import flip, color_aug
from utils.image import get_affine_transform, affine_transform
from utils.image import gaussian_radius, draw_umich_gaussian, draw_msra_gaussian
import pycocotools.coco as coco

In [20]:
index = 0
img_id = self.images[index]
img_info = self.coco.loadImgs(ids=[img_id])[0]
img_path = os.path.join(self.img_dir, img_info['file_name'])
img = cv2.imread(img_path)
if 'calib' in img_info:
  calib = np.array(img_info['calib'], dtype=np.float32)
else:
  calib = self.calib

In [21]:
height, width = img.shape[0], img.shape[1]
c = np.array([img.shape[1] / 2., img.shape[0] / 2.])
if self.opt.keep_res:
  s = np.array([self.opt.input_w, self.opt.input_h], dtype=np.int32)
else:
  s = np.array([width, height], dtype=np.int32)

aug = False
if self.split == 'train' and np.random.random() < self.opt.aug_ddd:
  aug = True
  sf = self.opt.scale
  cf = self.opt.shift
  s = s * np.clip(np.random.randn()*sf + 1, 1 - sf, 1 + sf)
  c[0] += img.shape[1] * np.clip(np.random.randn()*cf, -2*cf, 2*cf)
  c[1] += img.shape[0] * np.clip(np.random.randn()*cf, -2*cf, 2*cf)

trans_input = get_affine_transform(
  c, s, 0, [self.opt.input_w, self.opt.input_h])
inp = cv2.warpAffine(img, trans_input, 
                     (self.opt.input_w, self.opt.input_h),
                     flags=cv2.INTER_LINEAR)
inp = (inp.astype(np.float32) / 255.)
# if self.split == 'train' and not self.opt.no_color_aug:
#   color_aug(self._data_rng, inp, self._eig_val, self._eig_vec)
inp = (inp - self.mean) / self.std
inp = inp.transpose(2, 0, 1)

num_classes = self.opt.num_classes
trans_output = get_affine_transform(
  c, s, 0, [self.opt.output_w, self.opt.output_h])

hm = np.zeros(
  (num_classes, self.opt.output_h, self.opt.output_w), dtype=np.float32)
# vertex heatmap
vertex_hm = np.zeros(
  (9, self.opt.output_h, self.opt.output_w), dtype=np.float32)
# vertex coordinates from its center
vertex_coordinate = np.zeros((self.max_objs, 18), dtype=np.float32)
vertex_coordinate_mask = np.zeros((self.max_objs, 18), dtype=np.uint8)
wh = np.zeros((self.max_objs, 2), dtype=np.float32)
reg = np.zeros((self.max_objs, 2), dtype=np.float32)
dep = np.zeros((self.max_objs, 1), dtype=np.float32)
rotbin = np.zeros((self.max_objs, 2), dtype=np.int64)
rotres = np.zeros((self.max_objs, 2), dtype=np.float32)
dim = np.zeros((self.max_objs, 3), dtype=np.float32)
ind = np.zeros((self.max_objs), dtype=np.int64)
# precision lost in decline of resolution
vertex_reg_offset = np.zeros(
    (self.max_objs * 9, 2), dtype=np.float32)
vertex_ind = np.zeros((self.max_objs * 9), dtype=np.int64)
vertex_offfset_mask = np.zeros((self.max_objs * 9), dtype=np.int64)
reg_mask = np.zeros((self.max_objs), dtype=np.uint8)
rot_mask = np.zeros((self.max_objs), dtype=np.uint8)

ann_ids = self.coco.getAnnIds(imgIds=[img_id])
anns = self.coco.loadAnns(ids=ann_ids)
num_objs = min(len(anns), self.max_objs)
draw_gaussian = draw_msra_gaussian if self.opt.mse_loss else \
                draw_umich_gaussian
gt_det = []

In [22]:
for k in range(num_objs):
  ann = anns[k]
  bbox = self._coco_box_to_bbox(ann['bbox'])
  cls_id = int(self.cat_ids[ann['category_id']])
  if cls_id <= -99:
    continue
  # if flipped:
  #   bbox[[0, 2]] = width - bbox[[2, 0]] - 1
  #   使bbox的gt也会随着特征图缩小而缩小     
  bbox[:2] = affine_transform(bbox[:2], trans_output)
  bbox[2:] = affine_transform(bbox[2:], trans_output)
  bbox[[0, 2]] = np.clip(bbox[[0, 2]], 0, self.opt.output_w - 1)
  bbox[[1, 3]] = np.clip(bbox[[1, 3]], 0, self.opt.output_h - 1)
  h, w = bbox[3] - bbox[1], bbox[2] - bbox[0]
  if h > 0 and w > 0:
    radius = gaussian_radius((h, w))
    radius = max(0, int(radius))
    ct = np.array(
      [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
    ct_int = ct.astype(np.int32)
    if cls_id < 0:
      ignore_id = [_ for _ in range(num_classes)] \
                  if cls_id == - 1 else  [- cls_id - 2]
      if self.opt.rect_mask:
        hm[ignore_id, int(bbox[1]): int(bbox[3]) + 1, 
          int(bbox[0]): int(bbox[2]) + 1] = 0.9999
      else:
        for cc in ignore_id:
          draw_gaussian(hm[cc], ct, radius)
        hm[ignore_id, ct_int[1], ct_int[0]] = 0.9999
      continue
    draw_gaussian(hm[cls_id], ct, radius)


    for j in range(9):
      if 'vertex' in ann.keys():
        vertexes = np.array(ann['vertex'], dtype=np.float32)
        #  do affine transform for gt vertexes
        vertexes[j] = affine_transform(vertexes[j], trans_output)
        vertexes_int = vertexes.astype(np.int32)
        if vertexes[j,0] >= 0 and vertexes[j,0] <= self.opt.output_w and \
        vertexes[j,1] >= 0 and vertexes[j,1] <= self.opt.output_h :
          vertex_coordinate[k,j * 2: j * 2 + 2] = vertexes[j] - ct_int
          vertex_coordinate_mask[k, j * 2: j * 2 + 2] = 1
          vertex_reg_offset[k * 9 + j] = vertexes[j] - vertexes_int[j]
          vertex_ind[k * 9 +j] = vertexes_int[j,1] * self.opt.output_w + vertexes_int[j,0]
          vertex_offfset_mask[k * 9 +j] = 1
          # gaussian in vertexes heatmap
          draw_gaussian(vertex_hm[j], vertexes_int[j], radius)

    wh[k] = 1. * w, 1. * h
    gt_det.append([ct[0], ct[1], 1] + \
                  self._alpha_to_8(self._convert_alpha(ann['alpha'])) + \
                  [ann['depth']] + (np.array(ann['dim']) / 1).tolist() + [cls_id])
    if self.opt.reg_bbox:
      gt_det[-1] = gt_det[-1][:-1] + [w, h] + [gt_det[-1][-1]]
    # if (not self.opt.car_only) or cls_id == 1: # Only estimate ADD for cars !!!
    if 1:
      alpha = self._convert_alpha(ann['alpha'])
      # print('img_id cls_id alpha rot_y', img_path, cls_id, alpha, ann['rotation_y'])
      if alpha < np.pi / 6. or alpha > 5 * np.pi / 6.:
        rotbin[k, 0] = 1
        rotres[k, 0] = alpha - (-0.5 * np.pi)    
      if alpha > -np.pi / 6. or alpha < -5 * np.pi / 6.:
        rotbin[k, 1] = 1
        rotres[k, 1] = alpha - (0.5 * np.pi)
      dep[k] = ann['depth']
      dim[k] = ann['dim']
      # print('        cat dim', cls_id, dim[k])
      ind[k] = ct_int[1] * self.opt.output_w + ct_int[0]
      reg[k] = ct - ct_int
      reg_mask[k] = 1 if not aug else 0
      rot_mask[k] = 1
# print('gt_det', gt_det)
# print('')
#     ret = {'input': inp, 'hm': hm, 'dep': dep, 'dim': dim, 'ind': ind, 
#            'rotbin': rotbin, 'rotres': rotres, 'reg_mask': reg_mask,
#            'rot_mask': rot_mask}
ret = {'input': inp, 'hm': hm,'vertex_hm':vertex_hm,'vertex_coordinate':vertex_coordinate,
       'dep': dep, 'dim': dim, 'ind': ind, 'vertex_ind':vertex_ind,
       'vertex_coordinate_mask':vertex_coordinate_mask,
       'rotbin': rotbin, 'rotres': rotres, 'reg_mask': reg_mask,'vertex_offfset_mask':vertex_offfset_mask,
       'rot_mask': rot_mask}
if self.opt.reg_bbox:
  ret.update({'wh': wh})
if self.opt.reg_offset:
  ret.update({'reg': reg})
if self.opt.vertex_reg_offset:
  ret.update({'vertex_reg_offset': vertex_reg_offset})  
if self.opt.debug > 0 or not ('train' in self.split):
  gt_det = np.array(gt_det, dtype=np.float32) if len(gt_det) > 0 else \
           np.zeros((1, 18), dtype=np.float32)
  meta = {'c': c, 's': s, 'gt_det': gt_det, 'calib': calib,
          'image_path': img_path, 'img_id': img_id}
  ret['meta'] = meta

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

...

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
for u,v in ret.items():
    if type(v)!= dict:
        print(u,v.shape)
    else:
        for i ,j in v.items():
            print(i,j)

input (3, 384, 1280)
hm (3, 96, 320)
vertex_hm (9, 96, 320)
vertex_coordinate (50, 18)
dep (50, 1)
dim (50, 3)
ind (50,)
vertex_ind (450,)
vertex_coordinate_mask (50, 18)
rotbin (50, 2)
rotres (50, 2)
reg_mask (50,)
vertex_offfset_mask (450,)
rot_mask (50,)
wh (50, 2)
reg (50, 2)
vertex_reg_offset (450, 2)
c [621.  187.5]
s [1242  375]
gt_det [[ 1.0453333e+02  4.9244442e+01  1.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00
   1.0000000e+00  2.7559024e-01  9.6127522e-01  5.8490002e+01
   1.6700000e+00  1.8700000e+00  3.6900001e+00  9.3217316e+00
   5.5600662e+00  1.0000000e+00]
 [ 1.7592012e+02  4.5794525e+01  1.0000000e+00  0.0000000e+00
   1.0000000e+00 -7.9120889e-02  9.9686503e-01  0.0000000e+00
   0.0000000e+00  0.0000000e+00  1.0000000e+00  4.5840000e+01
   1.8600000e+00  6.0000002e-01  2.0200000e+00  3.1896973e+00
   7.7243156e+00  2.0000000e+00]]
calib [[7.215377e+02 0.000000e+00 6.095593e+02 4.485728e+01]
 [0.000000e+00 7.215377e+02 1.7

In [24]:
phase, epoch, data_loader = 'val', 0, val_loader
model_with_loss = trainer.model_with_loss
if phase == 'train':
      model_with_loss.train()
else:
  if len(trainer.opt.gpus) > 1:
    model_with_loss = trainer.model_with_loss.module
  model_with_loss.eval()
  torch.cuda.empty_cache()


ModelWithLoss(
  (model): ResNet_FP(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [25]:
iter_id = 0
batch = next(iter(data_loader))

In [26]:
for k in batch:
    if k != 'meta':
      batch[k] = batch[k].to(device=opt.device, non_blocking=True)    

In [27]:
for u,v in batch.items():
    if type(v)!= dict:
        print(u,v.shape)
    else:
        for i ,j in v.items():
            print(i,j)

input torch.Size([1, 3, 384, 1280])
hm torch.Size([1, 3, 96, 320])
vertex_hm torch.Size([1, 9, 96, 320])
vertex_coordinate torch.Size([1, 50, 18])
dep torch.Size([1, 50, 1])
dim torch.Size([1, 50, 3])
ind torch.Size([1, 50])
vertex_ind torch.Size([1, 450])
vertex_coordinate_mask torch.Size([1, 50, 18])
rotbin torch.Size([1, 50, 2])
rotres torch.Size([1, 50, 2])
reg_mask torch.Size([1, 50])
vertex_offfset_mask torch.Size([1, 450])
rot_mask torch.Size([1, 50])
wh torch.Size([1, 50, 2])
reg torch.Size([1, 50, 2])
vertex_reg_offset torch.Size([1, 450, 2])
c tensor([[621.0000, 187.5000]], dtype=torch.float64)
s tensor([[1242,  375]], dtype=torch.int32)
gt_det tensor([[[104.5333,  49.2444,   1.0000,   0.0000,   0.0000,   0.0000,   1.0000,
            0.0000,   1.0000,   0.2756,   0.9613,  58.4900,   1.6700,   1.8700,
            3.6900,   9.3217,   5.5601,   1.0000],
         [175.9201,  45.7945,   1.0000,   0.0000,   1.0000,  -0.0791,   0.9969,
            0.0000,   0.0000,   0.0000,   1.00

In [28]:
outputs = model(batch['input'])

/home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [39]:
kp = RegWeightedL1Loss()
hm_loss, dep_loss, rot_loss, dim_loss = 0, 0, 0, 0
wh_loss, off_loss = 0, 0
vertex_hm_loss,vertex_coordinate_loss, vertex_off_loss = 0, 0, 0
for s in range(opt.num_stacks):
  output = outputs[s]
  output['hm'] = _sigmoid(output['hm'])
  # add vertex loss
  output['vertex_hm'] = _sigmoid(output['vertex_hm'])
  output['dep'] = 1. / (output['dep'].sigmoid() + 1e-6) - 1.

  if opt.eval_oracle_dep:
    output['dep'] = torch.from_numpy(gen_oracle_map(
      batch['dep'].detach().cpu().numpy(), 
      batch['ind'].detach().cpu().numpy(), 
      opt.output_w, opt.output_h)).to(opt.device)

  hm_loss += self.crit(output['hm'], batch['hm']) / opt.num_stacks

  # add weights
  vertex_hm_loss += self.crit(output['vertex_hm'], batch['vertex_hm']) / opt.num_stacks

  vertex_coordinate_loss +=  kp(output['vertex_coordinate'],batch['vertex_coordinate_mask'],batch['ind'], batch['vertex_coordinate']) / opt.num_stacks

  if opt.dep_weight > 0:
    dep_loss += self.crit_reg(output['dep'], batch['reg_mask'],
                              batch['ind'], batch['dep']) / opt.num_stacks
  if opt.dim_weight > 0:
    dim_loss += self.crit_reg(output['dim'], batch['reg_mask'],
                              batch['ind'], batch['dim']) / opt.num_stacks
  if opt.rot_weight > 0:
    rot_loss += self.crit_rot(output['rot'], batch['rot_mask'],
                              batch['ind'], batch['rotbin'],
                              batch['rotres']) / opt.num_stacks
  if opt.reg_bbox and opt.wh_weight > 0:
    wh_loss += self.crit_reg(output['wh'], batch['rot_mask'],
                             batch['ind'], batch['wh']) / opt.num_stacks
  if opt.reg_offset and opt.off_weight > 0:
    off_loss += self.crit_reg(output['reg'], batch['rot_mask'],
                              batch['ind'], batch['reg']) / opt.num_stacks

  if opt.vertex_reg_offset and opt.vertex_off_weight > 0:
    vertex_off_loss += self.crit_reg(output['vertex_reg_offset'], batch['vertex_offfset_mask'],batch['vertex_ind'], batch['vertex_reg_offset']) / opt.num_stacks
loss = opt.hm_weight * hm_loss + opt.dep_weight * dep_loss + \
       opt.dim_weight * dim_loss + opt.rot_weight * rot_loss + \
       opt.wh_weight * wh_loss + opt.off_weight * off_loss + \
       vertex_hm_loss + vertex_coordinate_loss +  opt.vertex_off_weight * vertex_off_loss

loss_stats = {'loss': loss, 'hm_loss': hm_loss, 'dep_loss': dep_loss, 
              'dim_loss': dim_loss, 'rot_loss': rot_loss, 
              'wh_loss': wh_loss, 'off_loss': off_loss,
             'vertex_hm_loss': vertex_hm_loss,'vertex_coordinate_loss':vertex_coordinate_loss
              , 'vertex_off_loss': vertex_off_loss}

/home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [41]:
loss
loss_stats

tensor(28574.6074, device='cuda:0', grad_fn=<ThAddBackward>)

{'loss': tensor(28574.6074, device='cuda:0', grad_fn=<ThAddBackward>),
 'hm_loss': tensor(21466.5488, device='cuda:0', grad_fn=<AddBackward>),
 'dep_loss': tensor(2.0627, device='cuda:0', grad_fn=<AddBackward>),
 'dim_loss': tensor(0.0773, device='cuda:0', grad_fn=<AddBackward>),
 'rot_loss': tensor(2.4254, device='cuda:0', grad_fn=<AddBackward>),
 'wh_loss': tensor(0.2611, device='cuda:0', grad_fn=<AddBackward>),
 'off_loss': tensor(0.0254, device='cuda:0', grad_fn=<AddBackward>),
 'vertex_hm_loss': tensor(7100.9941, device='cuda:0', grad_fn=<AddBackward>),
 'vertex_coordinate_loss': tensor(2.4273, device='cuda:0', grad_fn=<AddBackward>),
 'vertex_off_loss': tensor(0.0193, device='cuda:0', grad_fn=<AddBackward>)}

In [30]:
output = outputs[0]
output['hm'] = _sigmoid(output['hm'])
output['dep'] = 1. / (output['dep'].sigmoid() + 1e-6) - 1.

In [28]:
off_loss += self.crit_reg(output['reg'], batch['rot_mask'],
                                  batch['ind'], batch['reg']) / opt.num_stacks

In [41]:
output['reg'].shape
batch['rot_mask'].shape
batch['ind'].shape
batch['reg'].shape
batch['vertex_ind'].shape

torch.Size([1, 2, 96, 320])

torch.Size([1, 50])

torch.Size([1, 50])

torch.Size([1, 50, 2])

torch.Size([1, 50, 9])

In [32]:
output['vertex_coordinate'].shape

torch.Size([1, 18, 96, 320])

In [31]:
if opt.vertex_reg_offset and opt.vertex_off_weight > 0:
        for i in range(0,18,2):
          vertex_off_loss += self.crit_reg(output['vertex_reg_offset'], batch['rot_mask'],
                  batch['vertex_ind'][:,:,int(i/2)], batch['vertex_reg_offset'][...,i:i+2]) / opt.num_stacks

In [34]:
output['dim'].shape
batch['reg_mask'].shape
batch['ind'].shape
batch['dim'].shape
output['vertex_hm'].shape
batch['vertex_hm'].shape

torch.Size([1, 3, 96, 320])

torch.Size([1, 50])

torch.Size([1, 50])

torch.Size([1, 50, 3])

torch.Size([1, 9, 96, 320])

torch.Size([1, 9, 96, 320])

In [30]:
vertex_hm_loss,vertex_coordinate_loss, vertex_off_loss = 0, 0, 0
vertex_hm_loss += self.crit(output['vertex_hm'], batch['vertex_hm']) / opt.num_stacks
      
for i in range(0,18,2):
    vertex_coordinate_loss +=  self.crit_reg(output['vertex_coordinate'][:,i:i+2],
    batch['reg_mask'],batch['vertex_ind'][:,:,int(i/2)],batch['vertex_coordinate'][...,i:i+2]) / opt.num_stacks

In [26]:
output, mask, ind, target = output['dep'], batch['reg_mask'],batch['ind'], batch['dep']

In [78]:
output.shape

torch.Size([1, 1, 96, 320])

In [ ]:
# pred = _transpose_and_gather_feat(output, ind)
# mask = mask.unsqueeze(2).expand_as(pred).float()
pred.shape
mask.shape
(pred * mask).shape
(target * mask).shape

In [32]:
output, loss, loss_stats = model_with_loss(batch)

In [33]:
loss_stats

{'loss': tensor(10078.5645, device='cuda:0', grad_fn=<ThAddBackward>),
 'hm_loss': tensor(7386.1650, device='cuda:0', grad_fn=<AddBackward>),
 'dep_loss': tensor(2.0511, device='cuda:0', grad_fn=<AddBackward>),
 'dim_loss': tensor(0.0773, device='cuda:0', grad_fn=<AddBackward>),
 'rot_loss': tensor(2.4254, device='cuda:0', grad_fn=<AddBackward>),
 'wh_loss': tensor(0.2611, device='cuda:0', grad_fn=<AddBackward>),
 'off_loss': tensor(0.0254, device='cuda:0', grad_fn=<AddBackward>),
 'vertex_hm_loss': tensor(2686.7683, device='cuda:0', grad_fn=<AddBackward>),
 'vertex_coordinate_loss': tensor(0.8530, device='cuda:0', grad_fn=<ThAddBackward>),
 'vertex_off_loss': tensor(0.1741, device='cuda:0', grad_fn=<ThAddBackward>)}

In [ ]:
for u,v in output.items():
    print(u,v.shape)

In [47]:
import torch
r = torch.rand(3,3)

In [49]:
r

tensor([[0.1943, 0.3623, 0.9429],
        [0.8342, 0.9788, 0.6533],
        [0.5609, 0.9389, 0.7679]])

In [48]:
r.view(3,1,3).expand(3,5,3)

tensor([[[0.1943, 0.3623, 0.9429],
         [0.1943, 0.3623, 0.9429],
         [0.1943, 0.3623, 0.9429],
         [0.1943, 0.3623, 0.9429],
         [0.1943, 0.3623, 0.9429]],

        [[0.8342, 0.9788, 0.6533],
         [0.8342, 0.9788, 0.6533],
         [0.8342, 0.9788, 0.6533],
         [0.8342, 0.9788, 0.6533],
         [0.8342, 0.9788, 0.6533]],

        [[0.5609, 0.9389, 0.7679],
         [0.5609, 0.9389, 0.7679],
         [0.5609, 0.9389, 0.7679],
         [0.5609, 0.9389, 0.7679],
         [0.5609, 0.9389, 0.7679]]])

In [1]:
r[...,::2]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.


NameError: name 'r' is not defined

In [7]:
import numpy as np

In [8]:
input2 = np.array([315.66729736328125,
215.83056640625,
280.6308288574219,
215.83221435546875,
313.33001708984375,
211.55223083496094,
344.8763122558594,
211.55088806152344,
315.66729736328125,
186.17977905273438,
280.6308288574219,
186.18028259277344,
313.33001708984375,
184.8531951904297,
344.8763122558594,
184.85276794433594,
314.4374084472656,
199.53054809570312])

In [11]:
np.any(input2[::2]>300)

True

In [4]:
input2[1::2]

array([215.83056641, 215.83221436, 211.55223083, 211.55088806,
       186.17977905, 186.18028259, 184.85319519, 184.85276794,
       199.5305481 ])

In [6]:
np.concatenate([input2[::2], input2[1::2]]) 

array([315.66729736, 280.63082886, 313.33001709, 344.87631226,
       315.66729736, 280.63082886, 313.33001709, 344.87631226,
       314.43740845, 215.83056641, 215.83221436, 211.55223083,
       211.55088806, 186.17977905, 186.18028259, 184.85319519,
       184.85276794, 199.5305481 ])

In [12]:
np.sum(np.abs(np.array([-2,3,4])))

9

In [21]:
a = np.array([1,2,3,4,5])
a[[0,3]]

array([1, 4])

In [15]:
a.ndim

1